In [20]:
import os
import pandas as pd
import numpy as np
from glob import glob

csv_combined = glob('*_combined_*.csv')
df_combined = [(f.split('_')[0] ,pd.read_csv(f)) for f in csv_combined]

ttest_combined = glob('*_ttest_*.csv')
df_ttest = {f.split('_')[0] : pd.read_csv(f) for f in ttest_combined}

html_content = ""
for (info,df) in df_combined:
    if info=='apathy':
        html_content += '<h2>Apathique vs non apathique</h2>\n'
    else:
        html_content += '<h2>Dépressif vs non dépressif</h2>\n'
    
    html_content += '<hr>\n'
    
    #Multiply float columns by 100 and round to 1 decimal place
    float_cols = df.select_dtypes(include=['float']).columns
    df[float_cols] = (df[float_cols] * 100).round(1)
    ttest=df_ttest[info]
    
    html_content += '<h3>Significativité des différences (test de Student) :</h3>\n'
    html_content += ttest[ttest['keep']][['feature','t','p']].to_html(index=False)
    html_content += '\n<br>\n'
    
    html_content += '<h3>Scores moyens (%) :</h3>\n'
    html_content += df[['model']+float_cols.tolist()].to_html(index=False)
    html_content += '\n<br><br>\n'

# Save to HTML file
with open('results_classif.html', 'w', encoding='utf-8') as f:
    f.write(html_content)


In [21]:
import os
import pandas as pd
import numpy as np
from glob import glob
csv_combined = glob('*_regression_*.csv')
df_combined = [(f.split('_')[0] ,pd.read_csv(f)) for f in csv_combined]

correlation_combined = glob('*_correlation_*.csv')
df_correlation = {f.split('_')[0] : pd.read_csv(f) for f in correlation_combined}
html_content = ""
for (info, df) in df_combined:
    html_content += f"<h2>{info.upper()} : corrélation avec les scores</h2>\n<hr>\n"
    
    # Multiply float columns by 100 and round to 1 decimal place
    float_cols = df.select_dtypes(include=['float']).columns
    df[float_cols] = df[float_cols].round(1)
    float_cols = ['mae', 'r2']
    df = df[df['feature_type'] == 'combined']
    corr = df_correlation[info]
    
    html_content += "<h3>Significativité des différences (test) :</h3>\n"
    html_content += corr[corr['significant_pearson']][['feature', 'pearson_corr', 'pearson_pval']].to_html(index=False)
    html_content += "\n<br>\n"
    
    html_content += "<h3>Scores moyens (%) :</h3>\n"
    html_content += df[['model'] + float_cols].to_html(index=False)
    html_content += "\n<br><br>\n"

# Save to HTML file
with open('results_regression.html', 'w', encoding='utf-8') as f:
    f.write(html_content)